## 1조 데이터 전처리 및 EDA 코드

In [1]:
# 패키지 로딩

# 전처리
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#폰트
import matplotlib
from matplotlib import font_manager, rc
import platform 
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)


#EDA
import klib
import pandas_profiling

#열 출력 개수 늘리기
pd.set_option("display.max_columns", 30)

In [3]:
# 데이터 읽어오기
data = pd.read_csv("데이터.csv", encoding = 'utf-8-sig')
data

,customer_id,created_at,pre_discount,post_discount,cashback_amount,discount_amount,total_promotion,push_permission,gender,is_foreigner,birthday,sign_up_date,merchant_id
0,207317,2020-03-22 10:27:52.498,36236,36236,1087,0,1087,True,female,False,1957-11-05,2019-11-30 16:48:19.339,9
1,95246,2020-03-12 09:41:09.845,12900,12900,387,0,387,True,female,False,1983-10-23,2019-07-09 08:21:14.098,6
2,208074,2020-02-01 08:54:53.246,7500,7500,225,0,225,True,female,False,1975-07-06,2019-08-15 01:02:13.336,9
3,15552,2020-03-06 10:23:21.183,11800,11800,354,0,354,False,male,False,1996-08-21,2019-08-06 02:09:28.712,4
4,511595,2020-03-13 12:00:27.914,29700,29700,891,0,891,True,female,False,1999-01-23,2020-03-12 00:33:46.147,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5623805,434398,2019-12-12 07:48:51.491,7547,7547,0,0,0,True,female,False,1975-02-06,2019-06-17 11:09:42.453,5
5623806,481515,2019-12-20 07:37:15.259,22900,20900,0,2000,2000,True,female,False,1989-01-23,2019-08-22 01:26:58.339,2
5623807,298192,2020-01-05 01:55:28.331,19790,18790,0,1000,1000,True,female,False,1985-04-06,2019-09-13 17:15:21.732,1
5623808,430962,2019-10-01 03:32:17.435,15921,14152,0,1769,1769,False,male,False,1983-07-25,2019-06-20 00:59:09.309,3


In [4]:
data['push_permission'] = data['push_permission'].apply(lambda x: 1 if x == True else -1) # 계산을 편하게 하기 위해 True = 1, False = -1로 변경
a = data.groupby('customer_id')[['push_permission']].sum().reset_index() # 고객별로 푸쉬알림에 대해 sum
b = data.groupby('customer_id')[['created_at']].count().reset_index() # 고객별로 건수에 대해 count
c = b.merge(a, how='left') # 조인
d = c['created_at'] - abs(c['push_permission']) # 건수 - sum(푸쉬알림)
d.sum()

0

In [10]:
idx = data[data['birthday'] == "1900-01-01"].index     # birthday == 1900-01-01인 인덱스의 값이 저장됨
idx

In [11]:
data.drop(idx , inplace=True) #제거

In [13]:
#2020년 기준 생년월일 계산
data['birthday'] = pd.to_datetime(data['birthday'], infer_datetime_format=True)
data['birthday_year'] = data['birthday'].dt.year
data['birthday_age'] = 2020 - data['birthday_year']

In [12]:
#저장
data.to_csv('데이터최종본.csv', index = False)

### 새 데이터프레임 생성

In [ ]:
#전처리한 데이터로 다시 불러오기
data = pd.read_csv("데이터최종본.csv", encoding = 'utf-8-sig')
data

In [14]:
#고객 ID별로 집계
ud = pd.DataFrame({'customer_id': data.customer_id.unique()})
ud = ud.sort_values(by=['customer_id'] ,ascending=True).reset_index(drop = True)
ud

,customer_id
0,1
1,2
2,3
3,4
4,5
...,...
541014,544917
541015,544918
541016,544919
541017,544920


### 고객별로 집계하여 붙이기

In [15]:
#성별
f = data.drop_duplicates(['customer_id'])[['customer_id','gender']]
f.rename(columns={'gender':'성별'},inplace=True)
ud = ud.merge(f, how='left')
ud

,customer_id,성별
0,1,female
1,2,female
2,3,male
3,4,male
4,5,female
...,...,...
541014,544917,female
541015,544918,female
541016,544919,female
541017,544920,male


In [16]:
#나이
f = data.drop_duplicates(['customer_id'])[['customer_id','birthday_age']]
f.rename(columns={'birthday_year':'나이'},inplace=True)
ud = ud.merge(f, how='left')
ud

,customer_id,성별,birthday_age
0,1,female,34
1,2,female,25
2,3,male,39
3,4,male,34
4,5,female,43
...,...,...,...
541014,544917,female,26
541015,544918,female,22
541016,544919,female,40
541017,544920,male,48


In [17]:
# 외국인 유무
f = data.drop_duplicates(['customer_id'])[['customer_id','is_foreigner']]
f.rename(columns={'is_foreigner':'외국인_유무'},inplace=True)
ud = ud.merge(f, how='left')
ud

In [18]:
#가입날짜
f = data.drop_duplicates(['customer_id'])[['customer_id','sign_up_date']]
f.rename(columns={'sign_up_date':'가입날짜'},inplace=True)
ud = ud.merge(f, how='left')
ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜
0,1,female,34,False,2019-08-25 18:28:34.815
1,2,female,25,False,2019-08-09 17:16:59.757
2,3,male,39,False,2019-08-24 16:43:13.693
3,4,male,34,False,2019-07-27 20:35:40.581
4,5,female,43,False,2020-01-11 12:28:23.167
...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713
541015,544918,female,22,False,2019-12-08 12:28:08.093
541016,544919,female,40,False,2019-08-03 16:06:57.3
541017,544920,male,48,False,2019-08-31 11:02:08.238


In [19]:
#푸시알림허용여부
f = data.drop_duplicates(['customer_id'])[['customer_id','push_permission']]
f.rename(columns={'push_permission':'푸시알림_허용여부'},inplace=True)
ud = ud.merge(f, how='left')
ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부
0,1,female,34,False,2019-08-25 18:28:34.815,False
1,2,female,25,False,2019-08-09 17:16:59.757,False
2,3,male,39,False,2019-08-24 16:43:13.693,True
3,4,male,34,False,2019-07-27 20:35:40.581,True
4,5,female,43,False,2020-01-11 12:28:23.167,True
...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False
541015,544918,female,22,False,2019-12-08 12:28:08.093,True
541016,544919,female,40,False,2019-08-03 16:06:57.3,True
541017,544920,male,48,False,2019-08-31 11:02:08.238,True


In [20]:
#이용횟수
f = data.groupby('customer_id')['discount_amount'].agg([
    ('이용횟수',np.size)
]).reset_index()
ud = ud.merge(f, how='left'); ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부,이용횟수
0,1,female,34,False,2019-08-25 18:28:34.815,False,2
1,2,female,25,False,2019-08-09 17:16:59.757,False,2
2,3,male,39,False,2019-08-24 16:43:13.693,True,1
3,4,male,34,False,2019-07-27 20:35:40.581,True,8
4,5,female,43,False,2020-01-11 12:28:23.167,True,6
...,...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False,2
541015,544918,female,22,False,2019-12-08 12:28:08.093,True,21
541016,544919,female,40,False,2019-08-03 16:06:57.3,True,43
541017,544920,male,48,False,2019-08-31 11:02:08.238,True,7


In [21]:
# 할인 받은 이용 횟수
f = data[data['total_promotion'] != 0].groupby('customer_id')['created_at'].count().reset_index()
f.rename(columns={'created_at':'할인_이용횟수'},inplace=True)
ud = ud.merge(f, how='left')
ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부,이용횟수,할인_이용횟수
0,1,female,34,False,2019-08-25 18:28:34.815,False,2,2.0
1,2,female,25,False,2019-08-09 17:16:59.757,False,2,2.0
2,3,male,39,False,2019-08-24 16:43:13.693,True,1,1.0
3,4,male,34,False,2019-07-27 20:35:40.581,True,8,6.0
4,5,female,43,False,2020-01-11 12:28:23.167,True,6,5.0
...,...,...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False,2,2.0
541015,544918,female,22,False,2019-12-08 12:28:08.093,True,21,15.0
541016,544919,female,40,False,2019-08-03 16:06:57.3,True,43,40.0
541017,544920,male,48,False,2019-08-31 11:02:08.238,True,7,6.0


In [22]:
#이용주기
f = data.groupby('customer_id')['created_at'].agg([
    ('이용주기', lambda x: int((x.astype('datetime64').max() - x.astype('datetime64').min()).days / x.str[:10].nunique()))
]).reset_index()
ud= ud.merge(f, how='left'); ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부,이용횟수,할인_이용횟수,이용주기
0,1,female,34,False,2019-08-25 18:28:34.815,False,2,2.0,0
1,2,female,25,False,2019-08-09 17:16:59.757,False,2,2.0,33
2,3,male,39,False,2019-08-24 16:43:13.693,True,1,1.0,0
3,4,male,34,False,2019-07-27 20:35:40.581,True,8,6.0,12
4,5,female,43,False,2020-01-11 12:28:23.167,True,6,5.0,12
...,...,...,...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False,2,2.0,14
541015,544918,female,22,False,2019-12-08 12:28:08.093,True,21,15.0,5
541016,544919,female,40,False,2019-08-03 16:06:57.3,True,43,40.0,8
541017,544920,male,48,False,2019-08-31 11:02:08.238,True,7,6.0,25


In [23]:
#이용업종수
f = data.groupby('customer_id').agg({
    'merchant_id': [('이용업종수', lambda x: x.nunique())],
})
f.columns = f.columns.droplevel()  
f=f.reset_index()
ud = ud.merge(f, how='left')
ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부,이용횟수,할인_이용횟수,이용주기,이용업종수
0,1,female,34,False,2019-08-25 18:28:34.815,False,2,2.0,0,2
1,2,female,25,False,2019-08-09 17:16:59.757,False,2,2.0,33,2
2,3,male,39,False,2019-08-24 16:43:13.693,True,1,1.0,0,1
3,4,male,34,False,2019-07-27 20:35:40.581,True,8,6.0,12,5
4,5,female,43,False,2020-01-11 12:28:23.167,True,6,5.0,12,4
...,...,...,...,...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False,2,2.0,14,2
541015,544918,female,22,False,2019-12-08 12:28:08.093,True,21,15.0,5,9
541016,544919,female,40,False,2019-08-03 16:06:57.3,True,43,40.0,8,10
541017,544920,male,48,False,2019-08-31 11:02:08.238,True,7,6.0,25,6


In [24]:
# 주 구매 업종 
f = data.groupby('customer_id')['merchant_id'].agg([('주구매업종', lambda x : x.value_counts().index[0])])
f=f.reset_index()
ud = ud.merge(f, how='left')
ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부,이용횟수,할인_이용횟수,이용주기,이용업종수,주구매업종
0,1,female,34,False,2019-08-25 18:28:34.815,False,2,2.0,0,2,6
1,2,female,25,False,2019-08-09 17:16:59.757,False,2,2.0,33,2,7
2,3,male,39,False,2019-08-24 16:43:13.693,True,1,1.0,0,1,4
3,4,male,34,False,2019-07-27 20:35:40.581,True,8,6.0,12,5,8
4,5,female,43,False,2020-01-11 12:28:23.167,True,6,5.0,12,4,6
...,...,...,...,...,...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False,2,2.0,14,2,6
541015,544918,female,22,False,2019-12-08 12:28:08.093,True,21,15.0,5,9,4
541016,544919,female,40,False,2019-08-03 16:06:57.3,True,43,40.0,8,10,3
541017,544920,male,48,False,2019-08-31 11:02:08.238,True,7,6.0,25,6,1


In [25]:
#날짜 데이터 형식으로 변경
data['created_at'] = pd.to_datetime(data['created_at'], format='%Y-%m-%d %H:%M:%S', errors='raise')
data['sign_up_date'] = pd.to_datetime(data['sign_up_date'], format='%Y-%m-%d %H:%M:%S', errors='raise')

In [26]:
# 주 이용 시간대
data['이용_시간대'] = data['created_at'].dt.hour
f = data.groupby('customer_id')['이용_시간대'].agg([('주이용시간대', lambda x : x.value_counts().index[0])])
f=f.reset_index()
ud = ud.merge(f, how='left')
ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부,이용횟수,할인_이용횟수,이용주기,이용업종수,주구매업종,주이용시간대
0,1,female,34,False,2019-08-25 18:28:34.815,False,2,2.0,0,2,6,10
1,2,female,25,False,2019-08-09 17:16:59.757,False,2,2.0,33,2,7,17
2,3,male,39,False,2019-08-24 16:43:13.693,True,1,1.0,0,1,4,16
3,4,male,34,False,2019-07-27 20:35:40.581,True,8,6.0,12,5,8,22
4,5,female,43,False,2020-01-11 12:28:23.167,True,6,5.0,12,4,6,12
...,...,...,...,...,...,...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False,2,2.0,14,2,6,1
541015,544918,female,22,False,2019-12-08 12:28:08.093,True,21,15.0,5,9,4,1
541016,544919,female,40,False,2019-08-03 16:06:57.3,True,43,40.0,8,10,3,10
541017,544920,male,48,False,2019-08-31 11:02:08.238,True,7,6.0,25,6,1,15


In [27]:
# 주 이용 요일
data['이용_요일'] = data['created_at'].dt.weekday
f = data.groupby('customer_id')['이용_요일'].agg([('주이용요일', lambda x : x.value_counts().index[0])])
f=f.reset_index()
ud = ud.merge(f, how='left') 
ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부,이용횟수,할인_이용횟수,이용주기,이용업종수,주구매업종,주이용시간대,주이용요일
0,1,female,34,False,2019-08-25 18:28:34.815,False,2,2.0,0,2,6,10,6
1,2,female,25,False,2019-08-09 17:16:59.757,False,2,2.0,33,2,7,17,1
2,3,male,39,False,2019-08-24 16:43:13.693,True,1,1.0,0,1,4,16,5
3,4,male,34,False,2019-07-27 20:35:40.581,True,8,6.0,12,5,8,22,0
4,5,female,43,False,2020-01-11 12:28:23.167,True,6,5.0,12,4,6,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False,2,2.0,14,2,6,1,6
541015,544918,female,22,False,2019-12-08 12:28:08.093,True,21,15.0,5,9,4,1,1
541016,544919,female,40,False,2019-08-03 16:06:57.3,True,43,40.0,8,10,3,10,0
541017,544920,male,48,False,2019-08-31 11:02:08.238,True,7,6.0,25,6,1,15,2


In [28]:
# 가입-첫 결제 소요 기간
data['가입_결제차이'] = (data['created_at'] - data['sign_up_date']).dt.days
f = data.groupby('customer_id')['가입_결제차이'].agg([('가입_첫결제차이', np.min)]).reset_index()
ud = ud.merge(f, how='left') 
ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부,이용횟수,할인_이용횟수,이용주기,이용업종수,주구매업종,주이용시간대,주이용요일,가입_첫결제차이
0,1,female,34,False,2019-08-25 18:28:34.815,False,2,2.0,0,2,6,10,6,0
1,2,female,25,False,2019-08-09 17:16:59.757,False,2,2.0,33,2,7,17,1,0
2,3,male,39,False,2019-08-24 16:43:13.693,True,1,1.0,0,1,4,16,5,-1
3,4,male,34,False,2019-07-27 20:35:40.581,True,8,6.0,12,5,8,22,0,67
4,5,female,43,False,2020-01-11 12:28:23.167,True,6,5.0,12,4,6,12,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False,2,2.0,14,2,6,1,6,0
541015,544918,female,22,False,2019-12-08 12:28:08.093,True,21,15.0,5,9,4,1,1,4
541016,544919,female,40,False,2019-08-03 16:06:57.3,True,43,40.0,8,10,3,10,0,-1
541017,544920,male,48,False,2019-08-31 11:02:08.238,True,7,6.0,25,6,1,15,2,0


In [29]:
a = data.groupby('customer_id')['created_at'].min().reset_index()
b= a.merge(data, how='left', on='created_at')
b = b[['customer_id_x','created_at','merchant_id']]
b.rename(columns={'customer_id_x':'customer_id','merchant_id':'첫결제_이용업종'},inplace=True)
b = b.drop_duplicates(['customer_id'])[['customer_id','첫결제_이용업종']]
ud = ud.merge(b, how='left') 
ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부,이용횟수,할인_이용횟수,이용주기,이용업종수,주구매업종,주이용시간대,주이용요일,가입_첫결제차이,첫결제_이용업종
0,1,female,34,False,2019-08-25 18:28:34.815,False,2,2.0,0,2,6,10,6,0,6
1,2,female,25,False,2019-08-09 17:16:59.757,False,2,2.0,33,2,7,17,1,0,9
2,3,male,39,False,2019-08-24 16:43:13.693,True,1,1.0,0,1,4,16,5,-1,4
3,4,male,34,False,2019-07-27 20:35:40.581,True,8,6.0,12,5,8,22,0,67,5
4,5,female,43,False,2020-01-11 12:28:23.167,True,6,5.0,12,4,6,12,1,-1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False,2,2.0,14,2,6,1,6,0,9
541015,544918,female,22,False,2019-12-08 12:28:08.093,True,21,15.0,5,9,4,1,1,4,6
541016,544919,female,40,False,2019-08-03 16:06:57.3,True,43,40.0,8,10,3,10,0,-1,3
541017,544920,male,48,False,2019-08-31 11:02:08.238,True,7,6.0,25,6,1,15,2,0,6


In [30]:
# 최근 사용 기간
f = (pd.datetime.now() - pd.DateOffset(years=1,months=11))
data['현재_구매일차이'] = (f - data['created_at']).dt.days
f = data.groupby('customer_id')['현재_구매일차이'].agg([('최근구매일', np.min)]).reset_index()
ud = ud.merge(f, how='left') 
ud

C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부,이용횟수,할인_이용횟수,이용주기,이용업종수,주구매업종,주이용시간대,주이용요일,가입_첫결제차이,첫결제_이용업종,최근구매일
0,1,female,34,False,2019-08-25 18:28:34.815,False,2,2.0,0,2,6,10,6,0,6,247
1,2,female,25,False,2019-08-09 17:16:59.757,False,2,2.0,33,2,7,17,1,0,9,197
2,3,male,39,False,2019-08-24 16:43:13.693,True,1,1.0,0,1,4,16,5,-1,4,249
3,4,male,34,False,2019-07-27 20:35:40.581,True,8,6.0,12,5,8,22,0,67,5,120
4,5,female,43,False,2020-01-11 12:28:23.167,True,6,5.0,12,4,6,12,1,-1,7,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False,2,2.0,14,2,6,1,6,0,9,234
541015,544918,female,22,False,2019-12-08 12:28:08.093,True,21,15.0,5,9,4,1,1,4,6,34
541016,544919,female,40,False,2019-08-03 16:06:57.3,True,43,40.0,8,10,3,10,0,-1,3,30
541017,544920,male,48,False,2019-08-31 11:02:08.238,True,7,6.0,25,6,1,15,2,0,6,66


In [31]:
#총 pre_거래금액, 평균 pre_거래금액, 최대 pre_거래금액, 최소 pre_거래금액
f = data.groupby('customer_id')['pre_discount'].agg([
    ('총pre_거래금액',np.sum),
    ('평균pre_거래금액', lambda x: np.round(np.mean(x))),
    ('최대pre_거래금액', np.max),
    ('최소pre_거래금액', np.min),
]).reset_index()
ud = ud.merge(f, how='left'); 
ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부,이용횟수,할인_이용횟수,이용주기,이용업종수,주구매업종,주이용시간대,주이용요일,가입_첫결제차이,첫결제_이용업종,최근구매일,총pre_거래금액,평균pre_거래금액,최대pre_거래금액,최소pre_거래금액
0,1,female,34,False,2019-08-25 18:28:34.815,False,2,2.0,0,2,6,10,6,0,6,247,24330,12165,14220,10110
1,2,female,25,False,2019-08-09 17:16:59.757,False,2,2.0,33,2,7,17,1,0,9,197,25030,12515,13530,11500
2,3,male,39,False,2019-08-24 16:43:13.693,True,1,1.0,0,1,4,16,5,-1,4,249,18120,18120,18120,18120
3,4,male,34,False,2019-07-27 20:35:40.581,True,8,6.0,12,5,8,22,0,67,5,120,227640,28455,106400,5310
4,5,female,43,False,2020-01-11 12:28:23.167,True,6,5.0,12,4,6,12,1,-1,7,37,65170,10862,13490,6000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False,2,2.0,14,2,6,1,6,0,9,234,41220,20610,24900,16320
541015,544918,female,22,False,2019-12-08 12:28:08.093,True,21,15.0,5,9,4,1,1,4,6,34,268665,12794,33800,3613
541016,544919,female,40,False,2019-08-03 16:06:57.3,True,43,40.0,8,10,3,10,0,-1,3,30,529022,12303,21800,4900
541017,544920,male,48,False,2019-08-31 11:02:08.238,True,7,6.0,25,6,1,15,2,0,6,66,462100,66014,198000,5900


In [32]:
# 총 post 거래 금액, 평균 post 거래 금액
f = data.groupby('customer_id')['post_discount'].agg([
    ('총post_거래금액',np.sum),
    ('평균post_거래금액', lambda x: np.round(np.mean(x))),
    ('최대post_거래금액', np.max),
    ('최소post_거래금액', np.min),
]).reset_index()
ud = ud.merge(f, how='left'); ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부,이용횟수,할인_이용횟수,이용주기,이용업종수,주구매업종,주이용시간대,주이용요일,가입_첫결제차이,첫결제_이용업종,최근구매일,총pre_거래금액,평균pre_거래금액,최대pre_거래금액,최소pre_거래금액,총post_거래금액,평균post_거래금액,최대post_거래금액,최소post_거래금액
0,1,female,34,False,2019-08-25 18:28:34.815,False,2,2.0,0,2,6,10,6,0,6,247,24330,12165,14220,10110,18960,9480,12640,6320
1,2,female,25,False,2019-08-09 17:16:59.757,False,2,2.0,33,2,7,17,1,0,9,197,25030,12515,13530,11500,19359,9680,9999,9360
2,3,male,39,False,2019-08-24 16:43:13.693,True,1,1.0,0,1,4,16,5,-1,4,249,18120,18120,18120,18120,13440,13440,13440,13440
3,4,male,34,False,2019-07-27 20:35:40.581,True,8,6.0,12,5,8,22,0,67,5,120,227640,28455,106400,5310,216980,27122,105400,4720
4,5,female,43,False,2020-01-11 12:28:23.167,True,6,5.0,12,4,6,12,1,-1,7,37,65170,10862,13490,6000,55170,9195,11490,6000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False,2,2.0,14,2,6,1,6,0,9,234,41220,20610,24900,16320,34740,17370,22900,11840
541015,544918,female,22,False,2019-12-08 12:28:08.093,True,21,15.0,5,9,4,1,1,4,6,34,268665,12794,33800,3613,252665,12032,31800,3613
541016,544919,female,40,False,2019-08-03 16:06:57.3,True,43,40.0,8,10,3,10,0,-1,3,30,529022,12303,21800,4900,483123,11235,20299,4900
541017,544920,male,48,False,2019-08-31 11:02:08.238,True,7,6.0,25,6,1,15,2,0,6,66,462100,66014,198000,5900,429100,61300,178000,5900


In [33]:
#총 할인 금액, 평균 할인 금액
f = data.groupby('customer_id')['discount_amount'].agg([
    ('총할인금액',np.sum),
    ('평균할인금액', lambda x: np.round(np.mean(x))),
]).reset_index()
ud = ud.merge(f, how='left'); ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부,이용횟수,할인_이용횟수,이용주기,이용업종수,주구매업종,주이용시간대,주이용요일,가입_첫결제차이,첫결제_이용업종,최근구매일,총pre_거래금액,평균pre_거래금액,최대pre_거래금액,최소pre_거래금액,총post_거래금액,평균post_거래금액,최대post_거래금액,최소post_거래금액,총할인금액,평균할인금액
0,1,female,34,False,2019-08-25 18:28:34.815,False,2,2.0,0,2,6,10,6,0,6,247,24330,12165,14220,10110,18960,9480,12640,6320,5370,2685
1,2,female,25,False,2019-08-09 17:16:59.757,False,2,2.0,33,2,7,17,1,0,9,197,25030,12515,13530,11500,19359,9680,9999,9360,5671,2836
2,3,male,39,False,2019-08-24 16:43:13.693,True,1,1.0,0,1,4,16,5,-1,4,249,18120,18120,18120,18120,13440,13440,13440,13440,4680,4680
3,4,male,34,False,2019-07-27 20:35:40.581,True,8,6.0,12,5,8,22,0,67,5,120,227640,28455,106400,5310,216980,27122,105400,4720,10660,1332
4,5,female,43,False,2020-01-11 12:28:23.167,True,6,5.0,12,4,6,12,1,-1,7,37,65170,10862,13490,6000,55170,9195,11490,6000,10000,1667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False,2,2.0,14,2,6,1,6,0,9,234,41220,20610,24900,16320,34740,17370,22900,11840,6480,3240
541015,544918,female,22,False,2019-12-08 12:28:08.093,True,21,15.0,5,9,4,1,1,4,6,34,268665,12794,33800,3613,252665,12032,31800,3613,16000,762
541016,544919,female,40,False,2019-08-03 16:06:57.3,True,43,40.0,8,10,3,10,0,-1,3,30,529022,12303,21800,4900,483123,11235,20299,4900,45899,1067
541017,544920,male,48,False,2019-08-31 11:02:08.238,True,7,6.0,25,6,1,15,2,0,6,66,462100,66014,198000,5900,429100,61300,178000,5900,33000,4714


In [34]:
#총 캐시백 금액, 평균 캐시백 금액
f = data.groupby('customer_id')['cashback_amount'].agg([
    ('총캐시백금액',np.sum),
    ('평균캐시백금액', lambda x: np.round(np.mean(x))),
]).reset_index()
ud = ud.merge(f, how='left'); ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부,이용횟수,할인_이용횟수,이용주기,이용업종수,주구매업종,주이용시간대,주이용요일,가입_첫결제차이,첫결제_이용업종,최근구매일,총pre_거래금액,평균pre_거래금액,최대pre_거래금액,최소pre_거래금액,총post_거래금액,평균post_거래금액,최대post_거래금액,최소post_거래금액,총할인금액,평균할인금액,총캐시백금액,평균캐시백금액
0,1,female,34,False,2019-08-25 18:28:34.815,False,2,2.0,0,2,6,10,6,0,6,247,24330,12165,14220,10110,18960,9480,12640,6320,5370,2685,0,0
1,2,female,25,False,2019-08-09 17:16:59.757,False,2,2.0,33,2,7,17,1,0,9,197,25030,12515,13530,11500,19359,9680,9999,9360,5671,2836,0,0
2,3,male,39,False,2019-08-24 16:43:13.693,True,1,1.0,0,1,4,16,5,-1,4,249,18120,18120,18120,18120,13440,13440,13440,13440,4680,4680,0,0
3,4,male,34,False,2019-07-27 20:35:40.581,True,8,6.0,12,5,8,22,0,67,5,120,227640,28455,106400,5310,216980,27122,105400,4720,10660,1332,0,0
4,5,female,43,False,2020-01-11 12:28:23.167,True,6,5.0,12,4,6,12,1,-1,7,37,65170,10862,13490,6000,55170,9195,11490,6000,10000,1667,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False,2,2.0,14,2,6,1,6,0,9,234,41220,20610,24900,16320,34740,17370,22900,11840,6480,3240,0,0
541015,544918,female,22,False,2019-12-08 12:28:08.093,True,21,15.0,5,9,4,1,1,4,6,34,268665,12794,33800,3613,252665,12032,31800,3613,16000,762,2296,109
541016,544919,female,40,False,2019-08-03 16:06:57.3,True,43,40.0,8,10,3,10,0,-1,3,30,529022,12303,21800,4900,483123,11235,20299,4900,45899,1067,177,4
541017,544920,male,48,False,2019-08-31 11:02:08.238,True,7,6.0,25,6,1,15,2,0,6,66,462100,66014,198000,5900,429100,61300,178000,5900,33000,4714,441,63


In [35]:
#결측값 0으로 처리
ud = ud.fillna(0)
#데이터 타입 변경
ud['할인_이용횟수'] = ud['할인_이용횟수'].astype("int64")

In [38]:
#데이터 확인
ud.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 541019 entries, 0 to 541018
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   customer_id   541019 non-null  int64 
 1   성별            541019 non-null  object
 2   birthday_age  541019 non-null  int64 
 3   외국인_유무        541019 non-null  bool  
 4   가입날짜          541019 non-null  object
 5   푸시알림_허용여부     541019 non-null  bool  
 6   이용횟수          541019 non-null  int64 
 7   할인_이용횟수       541019 non-null  int64 
 8   이용주기          541019 non-null  int64 
 9   이용업종수         541019 non-null  int64 
 10  주구매업종         541019 non-null  int64 
 11  주이용시간대        541019 non-null  int64 
 12  주이용요일         541019 non-null  int64 
 13  가입_첫결제차이      541019 non-null  int64 
 14  첫결제_이용업종      541019 non-null  int64 
 15  최근구매일         541019 non-null  int64 
 16  총pre_거래금액     541019 non-null  int64 
 17  평균pre_거래금액    541019 non-null  int64 
 18  최대pre_거래금액    541019 non

In [39]:
#데이터 저장
ud.to_csv('고객별_집계_데이터_u.csv', index = False, encoding = 'utf-8-sig')

## EDA

In [4]:
#데이터 불러오기
ud = pd.read_csv("고객별_집계_데이터_u.csv", encoding = 'utf-8-sig')
ud

,customer_id,성별,birthday_age,외국인_유무,가입날짜,푸시알림_허용여부,이용횟수,할인_이용횟수,이용주기,이용업종수,주구매업종,주이용시간대,주이용요일,가입_첫결제차이,첫결제_이용업종,최근구매일,총pre_거래금액,평균pre_거래금액,최대pre_거래금액,최소pre_거래금액,총post_거래금액,평균post_거래금액,최대post_거래금액,최소post_거래금액,총할인금액,평균할인금액,총캐시백금액,평균캐시백금액
0,1,female,34,False,2019-08-25 18:28:34.815,False,2,2,0,2,6,10,6,0,6,247,24330,12165,14220,10110,18960,9480,12640,6320,5370,2685,0,0
1,2,female,25,False,2019-08-09 17:16:59.757,False,2,2,33,2,7,17,1,0,9,197,25030,12515,13530,11500,19359,9680,9999,9360,5671,2836,0,0
2,3,male,39,False,2019-08-24 16:43:13.693,True,1,1,0,1,4,16,5,-1,4,249,18120,18120,18120,18120,13440,13440,13440,13440,4680,4680,0,0
3,4,male,34,False,2019-07-27 20:35:40.581,True,8,6,12,5,8,22,0,67,5,120,227640,28455,106400,5310,216980,27122,105400,4720,10660,1332,0,0
4,5,female,43,False,2020-01-11 12:28:23.167,True,6,5,12,4,6,12,1,-1,7,37,65170,10862,13490,6000,55170,9195,11490,6000,10000,1667,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541014,544917,female,26,False,2019-08-11 01:56:44.713,False,2,2,14,2,6,1,6,0,9,234,41220,20610,24900,16320,34740,17370,22900,11840,6480,3240,0,0
541015,544918,female,22,False,2019-12-08 12:28:08.093,True,21,15,5,9,4,1,1,4,6,34,268665,12794,33800,3613,252665,12032,31800,3613,16000,762,2296,109
541016,544919,female,40,False,2019-08-03 16:06:57.3,True,43,40,8,10,3,10,0,-1,3,30,529022,12303,21800,4900,483123,11235,20299,4900,45899,1067,177,4
541017,544920,male,48,False,2019-08-31 11:02:08.238,True,7,6,25,6,1,15,2,0,6,66,462100,66014,198000,5900,429100,61300,178000,5900,33000,4714,441,63


In [31]:
#상관계수 계산을 위한 전처리
ud['푸시알림_허용여부'] = ud['푸시알림_허용여부'].apply(lambda x: 1 if x == True else 0)
ud['성별'] = ud['성별'].apply(lambda x: 1 if x == 'female' else 0)
ud['외국인_유무'] = ud['외국인_유무'].apply(lambda x: 1 if x == True else 0)
ud = ud.drop('가입날짜', axis = 1)

In [32]:
#데이터 확인
ud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541019 entries, 0 to 541018
Data columns (total 27 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   customer_id   541019 non-null  int64
 1   성별            541019 non-null  int64
 2   birthday_age  541019 non-null  int64
 3   외국인_유무        541019 non-null  int64
 4   푸시알림_허용여부     541019 non-null  int64
 5   이용횟수          541019 non-null  int64
 6   할인_이용횟수       541019 non-null  int64
 7   이용주기          541019 non-null  int64
 8   이용업종수         541019 non-null  int64
 9   주구매업종         541019 non-null  int64
 10  주이용시간대        541019 non-null  int64
 11  주이용요일         541019 non-null  int64
 12  가입_첫결제차이      541019 non-null  int64
 13  첫결제_이용업종      541019 non-null  int64
 14  최근구매일         541019 non-null  int64
 15  총pre_거래금액     541019 non-null  int64
 16  평균pre_거래금액    541019 non-null  int64
 17  최대pre_거래금액    541019 non-null  int64
 18  최소pre_거래금액    541019 non-null  int64
 19  총p

In [ ]:
pr = data.profile_report()
pr.to_file('./profiling_report.html')

In [ ]:
#저장
pr1 = ud.profile_report()
pr1.to_file('./profiling_report.html')